In [14]:
import pandas
import pickle
import os
import pandas as pd
import nltk
import json
from datetime import datetime, timezone, date, time

In [15]:
STOCK_DATA_DIR = '../data/CHARTS/'
TIME_FRAMES = ['30-min', '1-hr', '4-hr', '1-day']

CURRENT_TIMEFRAME = '1-day'

In [16]:
# Train data on 1 hour

if CURRENT_TIMEFRAME == '1-day':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE1440.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON1440.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '4-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE240.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON240.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '1-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE60.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON60.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '30-min':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE30.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON30.csv'),names=['date','time','open','high','low','close','vol'])


In [17]:
# Label the stock price direction as -1, 1
for df in [aapl_stock_prices, amzn_stock_prices]:
    df['diff'] = (df['open'] <= df['close'])
    df['diff'] = df['diff'].apply(int)
    df['diff'].loc[df['diff']==0] = -1
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y.%m.%d').date())
    df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
    df['timestamp'] = df.apply(lambda x: datetime.combine(x['date'], x['time']), 1)
    df.sort_values(['timestamp'], axis=0, ascending=True, inplace=True)
    df.drop(columns=['date', 'time'], inplace=True)
    
aapl_stock_prices

/tmp/ipykernel_1310723/2582006541.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff'].loc[df['diff']==0] = -1
/tmp/ipykernel_1310723/2582006541.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff'].loc[df['diff']==0] = -1


,open,high,low,close,vol,diff,timestamp
0,93.159,94.194,93.028,93.460,2799,1,2014-06-30
1,93.973,94.546,93.611,93.993,4787,1,2014-07-01
2,94.344,94.515,93.540,93.902,4476,-1,2014-07-02
3,94.113,94.545,93.661,94.495,3454,1,2014-07-03
4,94.936,96.423,94.936,96.403,4638,1,2014-07-07
...,...,...,...,...,...,...,...
1138,156.250,156.490,153.650,155.920,62867,-1,2019-01-28
1139,156.470,161.620,150.980,160.540,66595,1,2019-01-29
1140,161.680,166.110,160.220,165.660,70344,1,2019-01-30
1141,166.100,168.980,164.530,166.770,62821,1,2019-01-31


In [22]:
# Load tokenized news data

with open('AaplExtractedTokens.pkl', 'rb') as f:
    tokenized_aapl_news_df = pickle.load(f)
    
with open('AmznExtractedTokens.pkl', 'rb') as f:
    tokenized_amzn_news_df = pickle.load(f)

In [23]:
tokenized_aapl_news_df

,Unnamed: 0,timestamp,source,tokens
0,0,2018-11-14 12:12:00,9to5mac.com,"['thing', 'kick', 'monday', 'news', 'ming', 'c..."
1,1,2018-11-05 00:00:00,morningstar.com,"['share', 'trade', 'monday', 'follow', 'maker'..."
2,2,2018-11-05 12:45:00,seekingalpha.com,"['portfolio', 'current', 'sit', 'core', 'divid..."
3,3,2018-11-26 00:00:00,morningstar.com,"['cristina', 'ker', 'implement', 'sever', 'boo..."
4,4,2018-11-23 00:00:00,zacks.com,"['trade', 'high', 'three', 'major', 'stock', '..."
...,...,...,...,...
73170,73170,2019-01-24 07:19:00,nasdaq.com,"['photo', 'top', 'tech', 'stock', 'higher', 'r..."
73171,73171,2019-01-14 20:18:00,moveefy.com,"['news', 'trio', 'babi', 'falcon', 'band', 'ca..."
73172,73172,2018-12-31 00:00:00,investing.com,"['major', 'index', 'other', 'group', 'amazon',..."
73173,73173,2019-01-28 19:00:00,yahoo.com,"['might', 'amazon', 'game', 'evan', 'motley', ..."


In [24]:
# Join news data and stock direction to label news
columns = ['news_timestamp', 'stock_timestamp', 'source', 'tokens', 'label']
labelled_aapl_news = pd.DataFrame(columns=columns)
labelled_amzn_news = pd.DataFrame(columns=columns)

skipped_cnt = 0
for index, row in tokenized_aapl_news_df.iterrows():
    try:
        matching_row = aapl_stock_prices.loc[aapl_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_aapl_news.loc[len(labelled_aapl_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d apple news articles with corresponding stock chart missing" % skipped_cnt)

skipped_cnt = 0
for index, row in tokenized_amzn_news_df.iterrows():
    try:
        matching_row = amzn_stock_prices.loc[amzn_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_amzn_news.loc[len(labelled_amzn_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d amazon news articles with corresponding stock chart missing" % skipped_cnt)

Skipped 1234 apple news articles with corresponding stock chart missing


In [26]:
# Save labelled news data

with open('AmznLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_amzn_news, f)
with open('AaplLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_aapl_news, f)

In [27]:
labelled_aapl_news

,news_timestamp,stock_timestamp,source,tokens,label
0,2018-11-14 12:12:00,2018-11-15,9to5mac.com,"['thing', 'kick', 'monday', 'news', 'ming', 'c...",1
1,2018-11-05 00:00:00,2018-11-06,morningstar.com,"['share', 'trade', 'monday', 'follow', 'maker'...",1
2,2018-11-05 12:45:00,2018-11-06,seekingalpha.com,"['portfolio', 'current', 'sit', 'core', 'divid...",1
3,2018-11-26 00:00:00,2018-11-27,morningstar.com,"['cristina', 'ker', 'implement', 'sever', 'boo...",1
4,2018-11-23 00:00:00,2018-11-26,zacks.com,"['trade', 'high', 'three', 'major', 'stock', '...",-1
...,...,...,...,...,...
71936,2019-01-24 07:19:00,2019-01-25,nasdaq.com,"['photo', 'top', 'tech', 'stock', 'higher', 'r...",1
71937,2019-01-14 20:18:00,2019-01-15,moveefy.com,"['news', 'trio', 'babi', 'falcon', 'band', 'ca...",1
71938,2018-12-31 00:00:00,2019-01-02,investing.com,"['major', 'index', 'other', 'group', 'amazon',...",-1
71939,2019-01-28 19:00:00,2019-01-29,yahoo.com,"['might', 'amazon', 'game', 'evan', 'motley', ...",1


In [13]:
labelled_amzn_news

,news_timestamp,stock_timestamp,source,tokens,label
0,2017-12-15 12:04:00,2017-12-18,seekingalpha.com,"['amazon', 'soon', 'sell', 'pull', 'e', 'platf...",1
1,2017-12-21 12:04:00,2017-12-22,seekingalpha.com,"['line', 'cashier', 'similar', 'fashion', 'fut...",-1
2,2017-12-21 12:27:00,2017-12-22,thestreet.com,"['fit', 'fang', 'stock', 'amazon', 'alphabet',...",-1
3,2017-12-22 00:00:00,2017-12-26,investors.com,"['amazon', 'cloud', 'china']",1
4,2017-12-22 20:57:00,2017-12-26,yahoo.com,"['stream', 'stick', 'plus', 'offer', 'play', '...",1
...,...,...,...,...,...
19911,2019-01-31 21:42:00,2019-02-01,nasdaq.com,"['also', 'prove', 'reach', 'mani', 'entertain'...",-1
19912,2019-01-31 22:03:00,2019-02-01,makemefeed.com,"['read', 'amazon', 'general', 'avail', 'publis...",-1
19913,2019-01-31 22:05:00,2019-02-01,yahoo.com,"['direct', 'cloud', 'amazon', 'unit', 'driven'...",-1
19914,2019-01-31 23:24:00,2019-02-01,marketwatch.com,"['earn', 'up', 'say', 'spend', 'real', 'test',...",-1
